In [73]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [74]:
txt_1 = 'Can I take your key?'

classifier(txt_1)

[{'label': 'NEGATIVE', 'score': 0.996327817440033}]

In [75]:
txt_2 = 'Neutral sentanse'

classifier(txt_2)

[{'label': 'NEGATIVE', 'score': 0.9839417338371277}]

In [76]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_99']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [77]:
inputs = tokenizer([txt_1, txt_2])

In [78]:
inputs

{'input_ids': [[101, 2064, 1045, 2202, 2115, 3145, 1029, 102], [101, 8699, 2741, 6962, 2063, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [79]:
inputs_with_padding = tokenizer([txt_1, txt_2], padding = True, truncation = True, max_length = 256, return_tensors="tf")

In [80]:
inputs_with_padding

{'input_ids': <tf.Tensor: shape=(2, 8), dtype=int32, numpy=
array([[ 101, 2064, 1045, 2202, 2115, 3145, 1029,  102],
       [ 101, 8699, 2741, 6962, 2063,  102,    0,    0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 8), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 0, 0]], dtype=int32)>}

In [81]:
logits = model(**inputs_with_padding).logits

In [82]:
logits

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 3.089482 , -2.5138037],
       [ 2.2197464, -1.895595 ]], dtype=float32)>

In [83]:
outputs = model(inputs_with_padding)

In [84]:
outputs

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 3.089482 , -2.5138037],
       [ 2.2197464, -1.895595 ]], dtype=float32)>, hidden_states=None, attentions=None)

In [85]:
import tensorflow as tf
predictions = tf.nn.softmax(outputs[0], axis=-1)
predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
model.config.id2label[predicted_class_id]

'NEGATIVE'

In [86]:
predictions

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.9963278 , 0.0036722 ],
       [0.98394173, 0.0160583 ]], dtype=float32)>